In [1]:
!pip install lightgbm==3.3.5
!pip install scikit-learn==1.0.2
!pip install joblib==1.2.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 3.1.1
    Uninstalling lightgbm-3.1.1:
      Successfully uninstalled lightgbm-3.1.1

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 12.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.1 MB/s eta 0:00:

In [2]:
#导入需要的包
import pandas as pd
import numpy as np
import shutil
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
from lightgbm import LGBMRegressor, early_stopping
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

# 调参

In [3]:
tur_id = 15
n = 2 - len(str(tur_id))
filename = '0' * n + str(tur_id) + '.csv'
data = pd.read_csv("区域赛训练集/" + filename)
data.dropna(subset=["YD15"], inplace=True)
data_median= data.median(axis=0)
data_mean = data.mean(axis=0)
data = data.fillna(value=data_median)
new_data = data.copy()
# 回归数据集自身特点，还是可以看中回归任务
train = new_data[["WINDSPEED", "PREPOWER", "WINDDIRECTION", "TEMPERATURE", "HUMIDITY", "PRESSURE", "ROUND(A.WS,1)"]]
print(train.shape)
target = new_data[['YD15']].values
# %%
x_train, x_test, y_train, y_test = train_test_split(train.values, target, test_size=0.1)

(43680, 7)


In [14]:
params = {    
  'boosting_type': 'gbdt', 
  'objective': 'regression', 
  'learning_rate': 0.1, 
  'num_leaves': 50, 
	'max_depth': 13,    
	'subsample': 0.8, 
  'colsample_bytree': 0.8, 
  }
data_train = lgb.Dataset(x_train, y_train, silent=True)
#lightgbm.cv官方文档https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.cv.html
cv_results = lgb.cv(
  params, 
  data_train, 
  num_boost_round=5000, 
  nfold=5, 
  stratified=False, 
  shuffle=True, 
  metrics='rmse',
  early_stopping_rounds=50, 
  verbose_eval=50, 
  show_stdv=True, 
  seed=0)
print('best n_estimators:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'][-1])
n_estimators = len(cv_results['rmse-mean'])
best_cv_score = cv_results['rmse-mean'][-1]


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000616 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1373
[LightGBM] [Info] Number of data points in the train set: 31448, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000628 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1373
[LightGBM] [Info] Number of data points in the train set: 31448, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1373
[LightGBM] [Info] Number of data points in the train set: 31448, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000576 seconds.
You can set `force_c

In [18]:
# 粗调
### 我们可以创建lgb的sklearn模型，使用上面选择的(学习率，评估器数目)
model_lgb = lgb.LGBMRegressor(objective='regression',
                              	metric='rmse',
								learning_rate=0.1,
							  	n_estimators=2764,
								num_leaves=50,
								max_depth=10,
								bagging_fraction = 0.8,
                              	feature_fraction = 0.8
								)
params_test1={    
			# 粗调
			# 'max_depth': range(12,15,1),     #16,17,18
			# 'num_leaves':range(59, 63, 1)   #56,57,58,59
			# 细调
			# 'max_depth': [18,19,20],
			# 'max_depth': [20,21,22],
			'max_depth': [24,25,26],
    		'num_leaves':[40,45,50,55,60,65,70,75]
			}
gsearch1 = GridSearchCV(estimator=model_lgb, 
						param_grid=params_test1, 
						scoring='neg_mean_squared_error', 
						cv=5, 	   #交叉验证参数，默认None，使用三折交叉验证。指定fold数量，默认为3，也可以是yield训练/测试数据的生成器。
						verbose=1, #日志冗长度，int：冗长度，0：不输出训练过程，1：偶尔输出，>1：对每个子模型都输出。
						n_jobs=-1  #并行数，-1即跟cpu核数一致
						)
gsearch1.fit(x_train, y_train)
means = gsearch1.cv_results_['mean_test_score']
params = gsearch1.cv_results_['params']
print(means)
print(params)
print(gsearch1.best_params_)
print(gsearch1.best_score_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[-7632977.68613635 -7594055.77468434 -7643476.9809466  -7556101.47513417
 -7677790.3729072  -7613744.81667507 -7589898.64954638 -7693042.46367455
 -7612578.08475129 -7595156.83035807 -7683005.74308361 -7565554.7877417
 -7649424.26929943 -7624776.07008912 -7597696.99998485 -7691508.98641834
 -7621063.33112753 -7601344.07435682 -7676412.82252979 -7574569.26306781
 -7642467.183074   -7607254.30830427 -7599137.70749841 -7673744.73370216]
[{'max_depth': 24, 'num_leaves': 40}, {'max_depth': 24, 'num_leaves': 45}, {'max_depth': 24, 'num_leaves': 50}, {'max_depth': 24, 'num_leaves': 55}, {'max_depth': 24, 'num_leaves': 60}, {'max_depth': 24, 'num_leaves': 65}, {'max_depth': 24, 'num_leaves': 70}, {'max_depth': 24, 'num_leaves': 75}, {'max_depth': 25, 'num_leaves': 40}, {'max_depth': 25, 'num_leaves': 45}, {'max_depth': 25, 'num_leaves': 50}, {'max_depth': 25, 'num_leaves': 55}, {'max_depth': 25, 'num_leaves': 60}, {'max_depth': 25, 'num_leaves': 65}, {'max_depth': 25, 'num_leaves': 70}, {'max_

In [ ]:
# # 细调
# import lightgbm as lgb
# from sklearn.model_selection import GridSearchCV
# params_test2={
#     # 'max_depth': range(13,20,3),
#     'max_depth': [14,15,16],
#     'num_leaves':[40,45,50,55,60,65,70,75]
# }
# gsearch2 = GridSearchCV(estimator=model_lgb, 
# 						param_grid=params_test2, 
# 						scoring='neg_mean_squared_error', 
# 						cv=5, verbose=1, n_jobs=4)
# gsearch2.fit(x_train, y_train)
# means = gsearch2.cv_results_['mean_test_score']
# params = gsearch2.cv_results_['params']
# print(means)
# print(params)
# print(gsearch2.best_params_)
# print(gsearch2.best_score_)


In [20]:
params_test3={
    # 'min_child_samples': [11,12,13,14,15],
    'min_child_samples': [15,16,17,18,19],
    'min_child_weight':[0.001, 0.002]
}
model_lgb = lgb.LGBMRegressor(objective='regression',
                                metric='rmse',
                                learning_rate=0.1,

                                n_estimators=3002,
                                num_leaves=55,
                                max_depth=24, 

                                bagging_fraction = 0.8,
                                feature_fraction = 0.8)
gsearch3 = GridSearchCV(estimator=model_lgb, 
                        param_grid=params_test3, 
                        scoring='neg_mean_squared_error', 
                        cv=5, verbose=1, n_jobs=-1)
gsearch3.fit(x_train, y_train)
means = gsearch3.cv_results_['mean_test_score']
params = gsearch3.cv_results_['params']
print(means)
print(params)
print(gsearch3.best_params_)
print(gsearch3.best_score_)



Fitting 5 folds for each of 10 candidates, totalling 50 fits
[-7431846.04075008 -7431846.04075008 -7537416.15046668 -7537416.15046668
 -7569977.92699377 -7569977.92699377 -7594254.41709847 -7594254.41709847
 -7543447.64131739 -7543447.64131739]
[{'min_child_samples': 15, 'min_child_weight': 0.001}, {'min_child_samples': 15, 'min_child_weight': 0.002}, {'min_child_samples': 16, 'min_child_weight': 0.001}, {'min_child_samples': 16, 'min_child_weight': 0.002}, {'min_child_samples': 17, 'min_child_weight': 0.001}, {'min_child_samples': 17, 'min_child_weight': 0.002}, {'min_child_samples': 18, 'min_child_weight': 0.001}, {'min_child_samples': 18, 'min_child_weight': 0.002}, {'min_child_samples': 19, 'min_child_weight': 0.001}, {'min_child_samples': 19, 'min_child_weight': 0.002}]
{'min_child_samples': 15, 'min_child_weight': 0.001}
-7431846.040750081


In [4]:
params_test4={
		    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
		    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
			}
model_lgb = lgb.LGBMRegressor(objective='regression',
								metric='rmse',

								learning_rate=0.1,
								n_estimators=3002, 
                             	num_leaves=55,
								max_depth=24, 
								min_child_samples=15,
								min_child_weight=0.001,

								bagging_freq = 5,
								)
gsearch4 = GridSearchCV(estimator=model_lgb, 
						param_grid=params_test4, 
						scoring='neg_mean_squared_error', 
						cv=5, verbose=1, n_jobs=-1)
gsearch4.fit(x_train, y_train)
means = gsearch4.cv_results_['mean_test_score']
params = gsearch4.cv_results_['params']
print(means)
print(params)
print(gsearch4.best_params_)
print(gsearch4.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will b

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6


...

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[-8807960.34471182 -8807960.34471182 -7850561.08699732 -7660403.3187588
 -7660403.3187588  -8640923.30592261 -8640923.30592261 -7744786.11489613
 -7573258.1425935  -7573258.1425935  -8598902.98701083 -8598902.98701083
 -7586309.29412097 -7482380.94629307 -7482380.94629307 -8485035.78269184
 -8485035.7

In [ ]:
params_test5={
	'bagging_fraction': [0.82, 0.85, 0.88, 0.9, 0.92, 0.95, 0.98 ],
	# 'bagging_fraction': [0.72, 0.75, 0.78, 0.8, 0.82, 0.85, 0.88 ],
    'feature_fraction': [0.72, 0.75, 0.78, 0.8, 0.82, 0.85, 0.88 ]
			}
model_lgb = lgb.LGBMRegressor(objective='regression',	
								metric='rmse',
								learning_rate=0.1, 
								n_estimators=2291, 
								num_leaves=45,
								max_depth=19, 
								min_child_samples=13,
								min_child_weight=0.001,
								)
gsearch5 = GridSearchCV(estimator=model_lgb, 
						param_grid=params_test5, 
						scoring='neg_mean_squared_error', 
						cv=5, verbose=1, n_jobs=-1)
gsearch5.fit(x_train, y_train)
# print(gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_)
means = gsearch5.cv_results_['mean_test_score']
params = gsearch5.cv_results_['params']
print(means)
print(params)
print(gsearch5.best_params_)
print(gsearch5.best_score_)

Fitting 5 folds for each of 49 candidates, totalling 245 fits


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82
[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82
[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82
[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] bagging_fraction is set=0.82, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82


...

[LightGBM] [Warning] feature_fraction is set=0.88, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.88
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.92, subsample=1.0 will be ignored. Current value: bagging_fraction=0.92


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] feature_fraction is set=0.72, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.72
[LightGBM] [Warning] bagging_fraction is set=0.92, subsample=1.0 will be ignored. Current value: bagging_fraction=0.92


In [11]:
params_test6={
    # 'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],
	'reg_alpha': [0.5, 0.6, 0.7],
    'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]
}
model_lgb = lgb.LGBMRegressor(objective='regression',
								metric='rmse',
								learning_rate=0.1, 
								n_estimators=2291, 
								num_leaves=45,
								max_depth=19, 
								min_child_samples=13,
								min_child_weight=0.001,
								bagging_fraction=1.0,
								feature_fraction=0.8,
								)
gsearch6 = GridSearchCV(estimator=model_lgb, 
						param_grid=params_test6, 
						scoring='neg_mean_squared_error', 
						cv=5, verbose=1, n_jobs=-1)
gsearch6.fit(x_train, y_train)

means = gsearch6.cv_results_['mean_test_score']
params = gsearch6.cv_results_['params']
print(means)
print(params)
print(gsearch6.best_params_)
print(gsearch6.best_score_)


Fitting 5 folds for each of 21 candidates, totalling 105 fits
[-13149787.19257294 -12995737.10637771 -13235965.59196066
 -13213349.88518679 -13064648.43315116 -13375627.41535364
 -13236979.27677801 -13175545.10091298 -13017756.08270633
 -13246183.70118383 -13251287.78668333 -13119654.13321174
 -13389944.63381493 -13270514.21247613 -13151528.00133388
 -12973602.05306796 -13298597.4092258  -13293823.44014619
 -13086002.81893501 -13350490.25216961 -13284974.22281039]
[{'reg_alpha': 0.5, 'reg_lambda': 0}, {'reg_alpha': 0.5, 'reg_lambda': 0.001}, {'reg_alpha': 0.5, 'reg_lambda': 0.01}, {'reg_alpha': 0.5, 'reg_lambda': 0.03}, {'reg_alpha': 0.5, 'reg_lambda': 0.08}, {'reg_alpha': 0.5, 'reg_lambda': 0.3}, {'reg_alpha': 0.5, 'reg_lambda': 0.5}, {'reg_alpha': 0.6, 'reg_lambda': 0}, {'reg_alpha': 0.6, 'reg_lambda': 0.001}, {'reg_alpha': 0.6, 'reg_lambda': 0.01}, {'reg_alpha': 0.6, 'reg_lambda': 0.03}, {'reg_alpha': 0.6, 'reg_lambda': 0.08}, {'reg_alpha': 0.6, 'reg_lambda': 0.3}, {'reg_alpha': 0.6

In [12]:
params = {
		    'boosting_type': 'gbdt', 
		    'objective': 'regression', 
		    'learning_rate': 0.05, 
		    'num_leaves': 45, 
		    'max_depth': 19,
			'min_child_samples': 13,   			#min_child_samples
			'min_child_weight': 0.001,			#min_child_weight
			'subsample': 1.0,					#bagging_fraction
			'colsample_bytree': 0.8,			#feature_fraction
			'reg_alpha': 0.7,					#reg_alpha
			'reg_lambda': 0.001,					#reg_lambd
    }
data_train = lgb.Dataset(x_train, y_train, silent=True)
cv_results = lgb.cv(params, data_train, 
					num_boost_round=50000, 
					nfold=5, 
					stratified=False, 
					shuffle=True, 
					metrics='rmse',    #均方误差
					early_stopping_rounds=50, 
					verbose_eval=100, 
					show_stdv=True)
print('best n_estimators:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'][-1])


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000410 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1168
[LightGBM] [Info] Number of data points in the train set: 16196, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000650 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1168
[LightGBM] [Info] Number of data points in the train set: 16196, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000790 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1168
[LightGBM] [Info] Number of data points in the train set: 16196, number of used features: 7
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000408 seconds.
You can set `force_c

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fractio

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[2000]	cv_agg's rmse: 3629.31 + 143.102
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fractio

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-v

[2100]	cv_agg's rmse: 3627.15 + 141.495
best n_estimators: 2121
best cv score: 3626.543099128544


In [ ]:
data = pd.read_csv("区域赛训练集/" + turb_id)
# data = pd.read_csv("区域赛训练集/" + filename)
data.dropna(subset=["YD15"], inplace=True)
data_mean = data.mean(axis=0)
data = data.fillna(value=data_mean)
new_data = data.copy()
train = new_data[
    ["WINDSPEED", "PREPOWER", "WINDDIRECTION", "TEMPERATURE", "HUMIDITY", "PRESSURE", "ROUND(A.WS,1)"]]
print(train.shape)
target = new_data[['YD15']].values
# %%
x_train, x_test, y_train, y_test = train_test_split(train.values, target, test_size=0.1)
print(type(x_train))
#定义模型 这里参数自行设定，获取不同调优结果，也可采用CV来调参
gbm = LGBMRegressor(
    boosting_type = 'gbdt', 
	objective = 'regression', 
	learning_rate = 0.01, 
    n_estimators = 2,
    n_jobs = -1,
    num_leaves = 50, 
    max_depth = 14,
	min_child_samples = 21,   			#min_child_samples
	min_child_weight = 0.001,	        #min_child_weight
	subsample = 1.0,					#bagging_fraction
	colsample_bytree = 0.8,		    	#feature_fraction
	reg_alpha = 0.01,					#reg_alpha
	reg_lambd = 0,						#reg_lambd
)
# 模型训练
# 指定训练集，验证集，采用l1正则化验证评估，10rounds训练损失没有下降停止，可以增至200~2000左右，这个本地训练更快一些
gbm = gbm.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric="l1",callbacks=[early_stopping(stopping_rounds=1000)])
# 模型预测
y_pre_gbm = gbm.predict(x_test)
# 模型评估
print(f"gbm {i} mean_absolute_error:{mean_squared_error(y_test, y_pre_gbm)}")
print(f"gbm {i} score:{gbm.score(x_test, y_test)}")
joblib.dump(gbm, 'model/gbm' + str(i) + '.pkl')